In [1]:
from CausalModule import CausalModule
# from run_effect_estimation import run_effect_estimation
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/anaconda3/envs/iocp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_url = "https://raw.githubusercontent.com/FenTechSolutions/CausalDiscoveryToolbox/master/cdt/data/resources/cyto_full_data.csv"
# data = pd.read_csv(data_url) 
import dowhy.datasets
dataset = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5,
        num_instruments = 2,
        num_treatments=1,
        num_samples=10000,
        treatment_is_binary=True,
        outcome_is_binary=True,
        stddev_treatment_noise=10)
data = dataset['df']
data['v0'] = data['v0'].astype(int)
data['y'] = data['y'].astype(int)
print(data.head())
print(dataset['treatment_name'])
print(dataset['outcome_name'])
discovery_algorithm = "pc-nl" 
treatment_variable = dataset['treatment_name'][0]
outcome_variable = dataset['outcome_name'][0]

treatment_value = 1
control_value = 0

    Z0        Z1        W0        W1        W2        W3        W4  v0  y
0  1.0  0.966938  1.532031  0.543733  1.079179 -1.034353 -0.368060   1  1
1  1.0  0.119280  0.256962  1.801282  2.142508  2.620849  0.200663   1  1
2  1.0  0.280015 -1.252065  0.871042  0.524455  0.943614  0.662023   1  1
3  0.0  0.861523  1.200008 -0.170130  0.254080  0.656970 -1.001795   1  1
4  1.0  0.610254  1.507466 -0.900992  1.399883 -1.225999 -1.918498   1  1
['v0']
y


In [3]:
print(treatment_variable)

v0


In [4]:
# Initialize the CausalModule with the provided parameters
causal_module = CausalModule(
    data=data,
    treatment_variable=treatment_variable,
    outcome_variable=outcome_variable,
)

# Find the causal graph
causal_module.find_causal_graph(algo=discovery_algorithm)
print(causal_module.graph.nodes())
# Refute the causal graph
# causal_module.refute_cgm(n_perm=10)
causal_module.see_graph()

# Create a causal graph model
causal_module.create_model()
# Identify the estimand
causal_module.identify_effect(method_name='default')
# Estimate the effect
causal_module.estimate_effect(ctrl_val=control_value, trtm_val=treatment_value)
# Refute the estimate
# causal_module.refute_estimate()

# Save results to CSV (default directory is 'outputs/results')
causal_module.store_results()

2025-09-03 05:47:17,816 INFO: CausalModule initialized with provided parameters.
2025-09-03 05:47:17,817 INFO: Finding causal graph using pc-nl algorithm
Depth=1, working on node 0:  11%|█         | 1/9 [00:00<00:00, 4899.89it/s]

KeyboardInterrupt: 

In [ ]:
causal_module.results.keys()

dict_keys(['graph_properties', 'node_quality_score', 'graph_quality_score', 'graph_quality_summary', 'effect_estimate'])

In [ ]:
causal_module.results['effect_estimate']

{'Effect Estimate': np.float64(0.5150567553712408),
 'Realized Estimand Expression': 'b: y~v0+W3+W1+v0*W2+v0*W4+v0*W0',
 'Treatment Value': 1,
 'Control Value': 0}